# Virtual Pruning

## Importing the digitized structure

In [1]:
from openalea.mtg import *
import numpy as np
from importlib import reload
import mangoG3; reload(mangoG3)
from mangoG3 import *

g = get_G3_mtg()

In [2]:
from matplotlib.pyplot import *
import mtgplot as mp
reload(mp)

<module 'mtgplot' from '/Users/fboudon/Develop/oagit/virtualpruning/notebooks/mtgplot.py'>

In [3]:
#sw = mp.plot_tree(g, leaves=True)
#sw


# Pruning

In [4]:
import pruning ; reload(pruning)
from pruning import *

In [5]:
prunedmtg = g # prune(g, 180)

In [6]:
intensity = continuous_intensity_from_pruned(prunedmtg)
print(intensity, intensity_level(intensity))

0.0 T0


In [27]:
prunedrepr = mp.representation(prunedmtg, wood = True, leaves=True, sensors=False)
print(len(prunedrepr))
print(len(prunedrepr.todict()))

34737
7925


In [8]:
#mp.display(prunedrepr)

SceneWidget(scenes=[{'id': '2NmTm2atmRrtKp9VDftxlvEq2', 'data': b'x\xda\x8c\x9d\x07x\x13G\xb7\x86M\xef\xa6\xf7…

# Drone simulation

In [28]:
from openalea.plantgl.all import *

def drone(scene, height = 600, resolution = 2):
    bbx = BoundingBox(scene)
    worldWidth = bbx.getXRange()
    worldheight = bbx.getYRange()
    up = (0,1,0)
    w, h = max(2,int(ceil(worldWidth/resolution))+1), max(2,int(ceil(worldheight/resolution))+1)
    print(w,h)
    z = ZBufferEngine(w,h,eColorBased)
    z.multithread = False
    minh = height-bbx.upperRightCorner.z
    maxh = height-bbx.lowerLeftCorner.z+minh
    print(worldWidth,worldheight, minh, maxh)
    z.setOrthographicCamera(-worldWidth/2., worldWidth/2., -worldheight/2., worldheight/2., minh , maxh)
    eyepos = bbx.getCenter()
    eyepos.z = height
    z.lookAt(eyepos, bbx.getCenter(), up) 
    z.process(scene)
    return z.grabZBufferPoints()
    

In [29]:
points,colors,ids = drone(prunedrepr)
print(len(points))

272 238
541.7516291377862 473.06886742523574 245.66585683123907 849.8495798387289
39652


In [30]:
mp.display(Scene([Shape(Translated(p,Sphere(1)),Material(c)) for p,c in zip(points,colors)]))

SceneWidget(scenes=[{'id': '1skOj2pyAUChSSk6eEravNnUZ', 'data': b'x\xda\x84\x9d\x07\x9c%U\xb1\xffG\xc9AY@X`\xd…